<a href="https://colab.research.google.com/github/CarlosMendez1997Col/GeoDatabases-And-Cloud-Computing-For-Water-Resources-Management/blob/main/1-Creation%20Geodatabase/Download_and_Geoprocessing_Databases_in_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and Geoprocessing Databases in Google Colab

---

> Water Resources Management using PostgreSQL and PgAdmin4

> Area of Interest (South America)

> Developed by MSc Carlos Mendez

MOST TABLES AND DATASETS USED:

1. South America Countries and Boundary [Url Data](https://international.ipums.org/international/gis.shtml)

2. First Level Administrative Units (FLAU) [Url Data](https://www.geoboundaries.org/globalDownloads.html)

3. Second Level Administrative Units (SLAU) [Url Data](https://www.geoboundaries.org/globalDownloads.html)

4. HydroSHEDS [Url Data](https://www.hydrosheds.org/products/hydrosheds)

5. HydroBASINS (Level 1 to 12) [Url Data](https://www.hydrosheds.org/products/hydrobasins)

6. HydroRIVERS [Url Data](https://www.hydrosheds.org/products/hydrorivers)

7. HydroLAKES [Url Data](https://www.hydrosheds.org/products/hydrolakes)

8. Global Lakes and Wetlands Database (GLWD) [Url Data](https://www.hydrosheds.org/products/glwd)

9. HydroWASTE [Url Data](https://www.hydrosheds.org/products/hydrowaste)

10. Global River Classification (GloRiC) [Url Data](https://www.hydrosheds.org/products/gloric)

11. Lake TEMP [Url Data](https://www.hydrosheds.org/products/laketemp)

12. Global Power Plant Database (GPPD) [Url Data](https://datasets.wri.org/datasets/global-power-plant-database)

## Install and import ArcGIS API for Python

In [109]:
# If you need to install any library, please delete commit and then install it
#!pip install arcgis
#!pip install geopandas
#!pip install rasterio
#!pip install shapely

## Import libraries and packages

In [110]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import xarray as xr
import matplotlib.pyplot as plt
import math
import zipfile
import os
import time
from datetime import datetime as dt
from osgeo import gdal, ogr, osr
from shapely.geometry import box

In [2]:
import arcgis
from arcgis.features import FeatureLayer, FeatureLayerCollection
from arcgis.geometry import SpatialReference

from google.colab import output
output.enable_custom_widget_manager()

# connect to GIS
from arcgis.gis import GIS

## Connect and Login in Arcgis Account

In [5]:
# Prompt user to provide username and password
import getpass
username = input('Enter username: ')
password = getpass.getpass("Enter your password: ")
gis = GIS("https://udistritalfjc.maps.arcgis.com/home", username, password)

Enter username: Camendezv_UDFJC
Enter your password: ··········


## Import and extract Databases in your local computer



### Connect to Google Drive

In [6]:
import os
os.makedirs('/content', exist_ok=True) # Create the parent directory if it doesn't exist

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Set Directory or WorkSpace
%cd /content/drive/MyDrive/Geodatabase

/content/drive/MyDrive/Geodatabase


### 1. South America Countries and Boundary (SACB)

In [8]:
!wget https://international.ipums.org/international/resources/gis/IPUMSI_world_release2024.zip

--2025-09-10 16:34:12--  https://international.ipums.org/international/resources/gis/IPUMSI_world_release2024.zip
Resolving international.ipums.org (international.ipums.org)... 128.101.163.176
Connecting to international.ipums.org (international.ipums.org)|128.101.163.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61705330 (59M) [application/zip]
Saving to: ‘IPUMSI_world_release2024.zip’

IPUMSI_world_releas 100%[===================>]  58.85M  27.7MB/s    in 2.1s    

2025-09-10 16:34:15 (27.7 MB/s) - ‘IPUMSI_world_release2024.zip’ saved [61705330/61705330]



In [9]:
!unzip "/content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.zip" -d "/content/drive/MyDrive/Geodatabase"

Archive:  /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.zip
 extracting: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.CPG  
  inflating: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.dbf  
  inflating: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.prj  
  inflating: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.sbn  
  inflating: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.sbx  
  inflating: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.shp  
  inflating: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.shp.xml  
  inflating: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.shx  


In [10]:
SACB = gpd.read_file('IPUMSI_world_release2024.shp')
SACB.head()

,OBJECTID,CNTRY_NAME,CNTRY_CODE,BPL_CODE,geometry
0,1,Algeria,012,13010.0,"MULTIPOLYGON (((-2.05592 35.0737, -2.05675 35...."
1,2,Angola,024,12010.0,"MULTIPOLYGON (((12.7976 -4.41685, 12.79875 -4...."
2,3,In dispute South Sudan/Sudan,9999,99999.0,"POLYGON ((28.08408 9.34722, 28.03889 9.34722, ..."
3,4,Benin,204,15010.0,"MULTIPOLYGON (((1.93753 6.30122, 1.93422 6.299..."
4,5,Botswana,072,14010.0,"POLYGON ((25.16312 -17.77816, 25.16383 -17.778..."


In [11]:
SACB.drop(['OBJECTID','CNTRY_CODE','BPL_CODE'], axis=1, inplace=True)
SACB.rename(columns={'CNTRY_NAME': 'Country'}, inplace=True)
SACB.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Country   285 non-null    object  
 1   geometry  285 non-null    geometry
dtypes: geometry(1), object(1)
memory usage: 4.6+ KB


In [12]:
SA_countries =  ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'French Guiana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela']
SACB_SA = SACB[SACB['Country'].isin(SA_countries)]

In [13]:
# If you want to verify the 14 countries of SA
#SACB_SA.head(14)

In [14]:
# If you want to display and visualize the 14 countries of SA
#SACB_SA.plot(column='Country', figsize=(16,8))

In [15]:
# Export data to Google Drive (.shp)
output_path_SACB = '/content/drive/MyDrive/Geodatabase/SA_Countries.shp'
SACB_SA.to_file(output_path_SACB)

In [16]:
## Delete original files (zip and .shp) to reduce space and volume
!rm '/content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.zip'

shapefile_prefix = '/content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024'

# List of common shapefile extensions
extensions = ['.CPG', '.dbf', '.prj', '.sbn', '.sbx', '.shp', '.shp.xml', '.shx']

for ext in extensions:
    file_path = shapefile_prefix + ext
    if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Deleted: {file_path}")
    else:
            print(f"File not found: {file_path}")

Deleted: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.CPG
Deleted: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.dbf
Deleted: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.prj
Deleted: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.sbn
Deleted: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.sbx
Deleted: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.shp
Deleted: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.shp.xml
Deleted: /content/drive/MyDrive/Geodatabase/IPUMSI_world_release2024.shx


### 2. First Level Administrative Units (FLAU)

In [17]:
!wget https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM1.zip

--2025-09-10 16:35:07--  https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM1.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/wmgeolab/geoBoundaries/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM1.zip [following]
--2025-09-10 16:35:07--  https://media.githubusercontent.com/media/wmgeolab/geoBoundaries/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM1.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103470246 (99M) [application/zip]
Saving to: ‘geoBoundariesCGAZ_ADM1.zip’

geoBoundariesCGAZ_A 100%[===================>]  98.68M  36.2MB/s    in 2

In [18]:
!unzip "/content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM1.zip" -d "/content/drive/MyDrive/Geodatabase"

Archive:  /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM1.zip
  inflating: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM1.shp  
  inflating: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM1.shx  
  inflating: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM1.dbf  
  inflating: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM1.prj  


In [19]:
FLAU = gpd.read_file('geoBoundariesCGAZ_ADM1.shp')
FLAU.head()

,shapeName,shapeID,shapeGroup,shapeType,geometry
0,Kandahar,12653393B40111500734429,AFG,ADM1,"POLYGON ((65.24153 32.2863, 65.72553 32.48037,..."
1,Zabul,12653393B56617740339660,AFG,ADM1,"POLYGON ((67.60666 31.44378, 67.60882 31.44909..."
2,Uruzgan,12653393B46006342616872,AFG,ADM1,"POLYGON ((66.27519 32.4255, 65.72553 32.48037,..."
3,Daykundi,12653393B78791504725813,AFG,ADM1,"POLYGON ((66.76157 33.25547, 66.38975 33.30701..."
4,Ghanzi,12653393B29313712249365,AFG,ADM1,"POLYGON ((68.06945 32.04564, 67.86439 32.14395..."


In [20]:
FLAU.drop(['shapeID','shapeGroup', 'shapeType'], axis=1, inplace=True)
FLAU.rename(columns={'shapeName': 'Department'}, inplace=True)
FLAU.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3224 entries, 0 to 3223
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Department  3224 non-null   object  
 1   geometry    3224 non-null   geometry
dtypes: geometry(1), object(1)
memory usage: 50.5+ KB


In [21]:
# Check CRSs
print(FLAU.crs)
print(SACB_SA.crs)

EPSG:4326
EPSG:4326


In [22]:
FLAU_Intersect = FLAU.overlay(SACB_SA, how='intersection')

/usr/local/lib/python3.12/dist-packages/geopandas/tools/overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 37572 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)


In [23]:
# If you want to check the Departments or States of SA countries
#FLAU_Intersect.head(30)

In [24]:
# If you want to display and visualize the data
#FLAU_Intersect.plot(column='Department', figsize=(16,8))

In [25]:
# Export data to Google Drive (.shp)
output_path_FLAU = '/content/drive/MyDrive/Geodatabase/SA_FLAU.shp'
FLAU_Intersect.to_file(output_path_FLAU)

In [26]:
## Delete Zip to reduce space and volume

!rm '/content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM1.zip'

shapefile_prefix = '/content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM1'

# List of common shapefile extensions
extensions = ['.dbf', '.prj', '.shp', '.shx']

for ext in extensions:
    file_path = shapefile_prefix + ext
    if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Deleted: {file_path}")
    else:
            print(f"File not found: {file_path}")

Deleted: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM1.dbf
Deleted: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM1.prj
Deleted: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM1.shp
Deleted: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM1.shx


### 3. Second Level Administrative Units (SLAU)

In [27]:
!wget https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM2.zip

--2025-09-10 16:37:53--  https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM2.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/wmgeolab/geoBoundaries/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM2.zip [following]
--2025-09-10 16:37:53--  https://media.githubusercontent.com/media/wmgeolab/geoBoundaries/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM2.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155911064 (149M) [application/zip]
Saving to: ‘geoBoundariesCGAZ_ADM2.zip’

geoBoundariesCGAZ_A 100%[===================>] 148.69M  47.7MB/s    in 

In [28]:
!unzip "/content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM2.zip" -d "/content/drive/MyDrive/Geodatabase"

Archive:  /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM2.zip
  inflating: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM2.shp  
  inflating: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM2.shx  
  inflating: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM2.dbf  
  inflating: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM2.prj  


In [29]:
SLAU = gpd.read_file('geoBoundariesCGAZ_ADM2.shp')
SLAU.head()

,shapeName,shapeID,shapeGroup,shapeType,geometry
0,Deh Bala,17698898B67359070524975,AFG,ADM2,"POLYGON ((70.51142 33.9492, 70.49439 33.94035,..."
1,Gulran,17698898B98443198567384,AFG,ADM2,"POLYGON ((62.0076 35.44597, 62.00902 35.39081,..."
2,Koshk,17698898B82675281335003,AFG,ADM2,"POLYGON ((61.99554 34.74465, 62.00493 34.75342..."
3,Chaparhar,17698898B74585757664988,AFG,ADM2,"POLYGON ((70.41933 34.23071, 70.41025 34.21431..."
4,Koshki Kohna,17698898B84066352785355,AFG,ADM2,"POLYGON ((62.41693 34.67412, 62.42883 34.73035..."


In [30]:
SLAU.drop(['shapeID','shapeGroup', 'shapeType'], axis=1, inplace=True)
SLAU.rename(columns={'shapeName': 'Municipality'}, inplace=True)
SLAU.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 49349 entries, 0 to 49348
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Municipality  49312 non-null  object  
 1   geometry      49349 non-null  geometry
dtypes: geometry(1), object(1)
memory usage: 771.2+ KB


In [31]:
SLAU_Intersect = SLAU.overlay(SACB_SA, how='intersection')

/usr/local/lib/python3.12/dist-packages/geopandas/tools/overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 32792 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)


In [32]:
# If you want to check the Departments or States of SA countries
#SLAU_Intersect.head(30)

In [33]:
# If you want to display and visualize the data
#SLAU_Intersect.plot(column='Municipality', figsize=(16,8))

In [34]:
# Export data to Google Drive (.shp)
output_path_SLAU = '/content/drive/MyDrive/Geodatabase/SA_SLAU.shp'
SLAU_Intersect.to_file(output_path_SLAU)

/tmp/ipython-input-3264475456.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  SLAU_Intersect.to_file(output_path_SLAU)
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Municipality' to 'Municipali'
  ogr_write(


In [35]:
## Delete Zip to reduce space and volume

!rm '/content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM2.zip'

shapefile_prefix = '/content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM2'

# List of common shapefile extensions
extensions = ['.dbf', '.prj', '.shp', '.shx']

for ext in extensions:
    file_path = shapefile_prefix + ext
    if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Deleted: {file_path}")
    else:
            print(f"File not found: {file_path}")

Deleted: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM2.dbf
Deleted: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM2.prj
Deleted: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM2.shp
Deleted: /content/drive/MyDrive/Geodatabase/geoBoundariesCGAZ_ADM2.shx


### 4. HydroSHEDS

#### Void Filled DEM

In [36]:
!wget https://data.hydrosheds.org/file/hydrosheds-v1-dem/hyd_sa_dem_30s.zip

--2025-09-10 17:38:24--  https://data.hydrosheds.org/file/hydrosheds-v1-dem/hyd_sa_dem_30s.zip
Resolving data.hydrosheds.org (data.hydrosheds.org)... 172.67.158.28, 104.21.14.61, 2606:4700:3036::ac43:9e1c, ...
Connecting to data.hydrosheds.org (data.hydrosheds.org)|172.67.158.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31834815 (30M) [application/zip]
Saving to: ‘hyd_sa_dem_30s.zip’

hyd_sa_dem_30s.zip  100%[===================>]  30.36M  56.1MB/s    in 0.5s    

2025-09-10 17:38:25 (56.1 MB/s) - ‘hyd_sa_dem_30s.zip’ saved [31834815/31834815]



In [37]:
!unzip "/content/drive/MyDrive/Geodatabase/hyd_sa_dem_30s.zip" -d "/content/drive/MyDrive/Geodatabase"

Archive:  /content/drive/MyDrive/Geodatabase/hyd_sa_dem_30s.zip
 extracting: /content/drive/MyDrive/Geodatabase/hyd_sa_dem_30s.tif  
 extracting: /content/drive/MyDrive/Geodatabase/HydroSHEDS_TechDoc_v1_4.pdf  


In [101]:
path_hydroSHEDS_DEM = '/content/drive/MyDrive/Geodatabase/hyd_sa_dem_30s.tif'
with rasterio.open(path_hydroSHEDS_DEM) as src:


  hydroSHEDS_DEM_SA = src.read(1)

In [106]:
'''
## If you want to plot or visualize the DEM of SA.
prof_hydroSHEDS_DEM = src.profile
print("Void Filled DEM Profile:", prof_hydroSHEDS_DEM)
plt.imshow(hydroSHEDS_DEM_SA, cmap='gray')
plt.title('Void Filled DEM')
plt.colorbar(label='Value')
plt.show()
'''

'\n## If you want to plot or visualize the DEM of SA.\nprof_hydroSHEDS_DEM = src.profile\nprint("Void Filled DEM Profile:", prof_hydroSHEDS_DEM)\nplt.imshow(hydroSHEDS_DEM_SA, cmap=\'gray\')\nplt.title(\'Void Filled DEM\')\nplt.colorbar(label=\'Value\')\nplt.show()\n'

In [ ]:
proj =

#### Flow Direction

In [ ]:
https://data.hydrosheds.org/file/hydrosheds-v1-dir/hyd_sa_dir_30s.zip

#### Flow Accumulation

In [ ]:
https://data.hydrosheds.org/file/hydrosheds-v1-acc/hyd_sa_acc_30s.zip

#### Flow Length Upstream

In [ ]:
https://data.hydrosheds.org/file/hydrosheds-v1-lup/hyd_sa_lup_15s.zip

#### Flow Length Downstream

In [ ]:
https://data.hydrosheds.org/file/hydrosheds-v1-ldn/hyd_sa_ldn_15s.zip

### 5. HydroBASINS (Level 1 to 12)

### 6. HydroRIVERS

### 7. HydroLAKES

### 8. Global Lakes and Wetlands Database (GLWD)

### 9. HydroWASTE

### 10. Global River Classification (GloRiC)

### 11. LakeTEMP

### 12. Global Power Plant Database (GPPD)

geoBoundariesCGAZ_ADM0.zip


In [ ]:
!rm -r /content/

In [ ]:
https://data.hydrosheds.org/file/hydrosheds-v1-dem/hyd_sa_dem_30s.zip


https://data.hydrosheds.org/file/hydrobasins/standard/hybas_sa_lev01-12_v1c.zip


https://data.hydrosheds.org/file/HydroRIVERS/HydroRIVERS_v10_sa_shp.zip


https://data.hydrosheds.org/file/hydrolakes/HydroLAKES_polys_v10_shp.zip


https://figshare.com/ndownloader/files/54001748


https://figshare.com/ndownloader/files/31910714


https://data.hydrosheds.org/file/hydrosheds-associated/gloric/GloRiC_v10_shapefile.zip


https://figshare.com/ndownloader/files/46397785


https://datasets.wri.org/private-admin/dataset/53623dfd-3df6-4f15-a091-67457cdb571f/resource/66bcdacc-3d0e-46ad-9271-a5a76b1853d2/download/globalpowerplantdatabasev130.zip


!wget your_external_url_to_zip_file.zip